In [ ]:
import os 
import numpy as np
import librosa 
import json
import cv2

def get_file_descriptors(dirpath):
        directory={}
        dataset_path=dirpath
        cwd= os.getcwd()
        for i , (dirpath, dirname, filename) in enumerate(os.walk(dataset_path)):
            if(dirpath!=dataset_path):
                dirname=dirpath.split("/")[-1]
                files={}
                file_list=[]
                index=0
                for file in filename:
                    filepath = os.path.join( dirpath, file)
                    if ( (filepath.endswith('.wav'))):
                        if(file.startswith('.')):
                            pass
                        else:
                            file_list.append(filepath)
                file_list.sort()
                if(len(file_list)>0):
                    for filepath in file_list:
                        files[index]=filepath
                        index+=1
                    directory[dirname]=files
        return directory


In [ ]:
def wavtospec(indx):
    fft_len=318
    frames=[]
    #duration in milliseconds
    
    return frames

In [ ]:
def wavtoimg(filepath):
    fft_len=318
    frames=[]
    #duration in milliseconds
    block_size=310
    tot_duration=62000
    for i in range(0,tot_duration,block_size):
        x,sr=librosa.load(filepath,duration=block_size/1000,offset=i/1000)
        x=librosa.resample(x, orig_sr=sr, target_sr=8000)
        stft_x=librosa.stft(x, n_fft=fft_len, hop_length=fft_len//4, win_length=fft_len, window='hann', center=True, dtype=None, pad_mode='constant')
        #print(stft_x.shape)
        frames.append(stft_x)
    return frames


def tagimage(x,outfile_path):
    mag,phase=librosa.magphase(x,power=1)
    mag = 255*mag
    mag = mag.astype(np.uint8)
    mag = np.flip(mag,axis=0)
    cv2.imwrite(outfile_path,mag)

def generateDataset(input_dir,outputdir):
    directory=get_file_descriptors(input_dir)
    spec_dict={}
    for key in directory.keys():
        spec_dict[key]={}
        k=0
        print(key)
        for indx,filepath in enumerate(directory[key].values()):
            frames=wavtoimg(filepath)
            for j,y in enumerate(frames):
                filename=key+"_fileid_"+str(indx)+"_"+str(j)+"_.jpg"
                out_file=os.path.join(outputdir,filename)
                tagimage(y,out_file)
                spec_dict[key][k]=out_file
                k+=1
    json_path="./daps_dict_unet2.json"
    json_directory = json.dumps(spec_dict,indent=4)
    with open(json_path, "w") as outfile:
        outfile.write(json_directory)

In [ ]:
in_dir_path= "../../dataset_daps/daps"
generateDataset(in_dir_path,"../../dataset_daps/daps/data_unet")